<a href="https://colab.research.google.com/github/vishnuvryeruva/Data-Augmentation--Activation-functions--Regularization/blob/main/Weights_and_Biases_of_Hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb --upgrade

# workaround to fetch MNIST data
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [3]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Defining the Sweep

In [4]:
sweep_config = {
    'method': 'random'
    }# can use grid, bayseian

In [5]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

#Naming hyperparameters
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 1}
    })

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

In [6]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


Initialize the sweep

In [7]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: nvw7r4at
Sweep URL: https://wandb.ai/team-dragonites/pytorch-sweeps-demo/sweeps/nvw7r4at


Run sweep Agent

In [8]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})           

In [9]:
def build_dataset(batch_size):
   
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST(".", train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)

    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

In [10]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: dn9n0pom with config:
wandb: 	batch_size: 216
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.004686425934782634
wandb: 	optimizer: sgd
wandb: Currently logged in as: team-dragonites (use `wandb login --relogin` to force relogin)


batch loss,█████▇▇▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁
epoch,▁
loss,▁
batch loss,0.5679
epoch,0
loss,1.37368


wandb: Agent Starting Run: b8mlyqxq with config:
wandb: 	batch_size: 136
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.06762591729976547
wandb: 	optimizer: adam


batch loss,▁█▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,1.84439
epoch,0
loss,2.91937


wandb: Agent Starting Run: 1oigfd9k with config:
wandb: 	batch_size: 72
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.09069036351621584
wandb: 	optimizer: sgd


batch loss,█▅▃▃▄▂▃▂▂▃▂▃▂▂▃▂▄▄▃▃▄▄▂▃▂▂▃▂▁▃▂▃▃▂▂▂▂▁▂▁
epoch,▁
loss,▁
batch loss,0.69935
epoch,0
loss,0.6472


wandb: Agent Starting Run: o5bcqww7 with config:
wandb: 	batch_size: 224
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.056643810983054925
wandb: 	optimizer: sgd


batch loss,██▇▆▅▄▂▂▂▃▂▃▂▁▂▂▂▃▂▂▃▂▁▂▁▂▂▂▁▁▂▂▁▂▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,0.36073
epoch,0
loss,0.63804


wandb: Agent Starting Run: qvffl75d with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.037374260596903346
wandb: 	optimizer: adam


batch loss,█▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,1.39999
epoch,0
loss,1.60972


Performing Sweep using Keras

In [11]:
import random

for run in range(5):
  wandb.init(

      project="basic-intro", 
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",})
  
  offset = random.random() / 5
  for ii in range(2, 10):
      acc = 1 - 2 ** -ii - random.random() / ii - offset
      loss = 2 ** -ii + random.random() / ii + offset
      wandb.log({"acc": acc, "loss": loss})
      
  wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▄▁█▅▇███
loss,█▃▃▂▁▂▂▁
acc,0.85474
loss,0.10192


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁▂▇▇▇█▇█
loss,▆█▅▂▂▁▁▁
acc,0.89808
loss,0.0703


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁▂▄█▇▆██
loss,▇█▁▁▄▂▁▂
acc,0.91751
loss,0.09286


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁▄▆▆▇▇▇█
loss,█▅▃▃▂▂▂▁
acc,0.85171
loss,0.124


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁▆▅█████
loss,█▃▃▂▃▁▁▂
acc,0.91158
loss,0.19143


In [12]:
%wandb stacey/deep-drive/runs/1wyssjcx -h 720

In [13]:
import random

import numpy as np
import tensorflow as tf
from wandb.keras import WandbCallback

# Simple Keras Model

# Launch 20 experiments, trying different dropout rates
for run in range(20):
  # Start a run, tracking hyperparameters
  wandb.init(
      project="keras-intro",

      config={
          "layer_1": 512,
          "activation_1": "relu",
          "dropout": random.uniform(0.01, 0.80),
          "layer_2": 10,
          "activation_2": "softmax",
          "optimizer": "sgd",
          "loss": "sparse_categorical_crossentropy",
          "metric": "accuracy",
          "epoch": 6,
          "batch_size": 256
      })
  config = wandb.config

  # Get the data
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train / 255.0, x_test / 255.0
  x_train, y_train = x_train[::5], y_train[::5] 
  x_test, y_test = x_test[::20], y_test[::20]
  labels = [str(digit) for digit in range(np.max(y_train) + 1)]

  model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
      tf.keras.layers.Dropout(config.dropout),
      tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
      ])

  model.compile(optimizer=config.optimizer,
                loss=config.loss,
                metrics=[config.metric]
                )

  logging_callback = WandbCallback(log_evaluation=True)

  history = model.fit(x=x_train, y=y_train,
                      epochs=config.epoch,
                      batch_size=config.batch_size,
                      validation_data=(x_test, y_test),
                      callbacks=[logging_callback]
                      )
  wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/6
300/300 [==============================] - 3s 3ms/step - loss: 0.4423 - accuracy: 0.8673 - val_loss: 0.2579 - val_accuracy: 0.9160 - _timestamp: 1649985286.0000 - _runtime: 26.0000
Epoch 2/6
300/300 [==============================] - 1s 3ms/step - loss: 0.2001 - accuracy: 0.9427 - val_loss: 0.2230 - val_accuracy: 0.9240 - _timestamp: 1649985287.0000 - _runtime: 27.0000
Epoch 3/6
300/300 [==============================] - 1s 3ms/step - loss: 0.1396 - accuracy: 0.9599 - val_loss: 0.1963 - val_accuracy: 0.9300 - _timestamp: 1649985288.0000 - _runtime: 28.0000
Epoch 4/6
300/300 [==============================] - 1s 3ms/step - loss: 0.1036 - accuracy: 0.9706 - val_loss: 0.1829 - val_accuracy: 0.9260 - _timestamp: 1649985288.0000 - _runtime: 28.0000
Epoch 5/6
300/300 [==============================] - 1s 3ms/step - loss: 0.0845 - accuracy: 0.9754 - val_loss: 0.1703 - val_accuracy: 0.9420 - _timestamp: 1649985289.0000 

accuracy,▁▆▇▇██
epoch,▁▂▄▅▇█
loss,█▄▂▂▁▁
val_accuracy,▁▃▄▃▇█
val_loss,█▆▄▃▂▁
accuracy,0.9835
best_epoch,5
best_val_loss,0.15971
epoch,5
loss,0.06302
val_accuracy,0.948


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 1/6
300/300 [==============================] - 1s 3ms/step - loss: 0.4349 - accuracy: 0.8742 - val_loss: 0.2631 - val_accuracy: 0.9160 - _timestamp: 1649985303.0000 - _runtime: 6.0000
Epoch 2/6
300/300 [==============================] - 1s 3ms/step - loss: 0.2039 - accuracy: 0.9413 - val_loss: 0.2201 - val_accuracy: 0.9200 - _timestamp: 1649985304.0000 - _runtime: 7.0000
Epoch 3/6
300/300 [==============================] - 1s 3ms/step - loss: 0.1413 - accuracy: 0.9591 - val_loss: 0.2117 - val_accuracy: 0.9320 - _timestamp: 1649985305.0000 - _runtime: 8.0000
Epoch 4/6
300/300 [==============================] - 1s 2ms/step - loss: 0.1044 - accuracy: 0.9698 - val_loss: 0.1534 - val_accuracy: 0.9460 - _timestamp: 1649985306.0000 - _runtime: 9.0000
Epoch 5/6
300/300 [==============================] - 1s 3ms/step - loss: 0.0806 - accuracy: 0.9772 - val_loss: 0.1466 - val_accuracy: 0.9520 - _timestamp: 1649985306.0000 - _runtime: 9.0000
Epoch 6/6
300/300 [==============================]

...

Epoch 1/6
300/300 [==============================] - 1s 3ms/step - loss: 0.4455 - accuracy: 0.8664 - val_loss: 0.3019 - val_accuracy: 0.9060 - _timestamp: 1649986010.0000 - _runtime: 6.0000
Epoch 2/6
300/300 [==============================] - 1s 3ms/step - loss: 0.2113 - accuracy: 0.9391 - val_loss: 0.2180 - val_accuracy: 0.9340 - _timestamp: 1649986011.0000 - _runtime: 7.0000
Epoch 3/6
300/300 [==============================] - 1s 3ms/step - loss: 0.1450 - accuracy: 0.9578 - val_loss: 0.2064 - val_accuracy: 0.9260 - _timestamp: 1649986011.0000 - _runtime: 7.0000
Epoch 4/6
300/300 [==============================] - 1s 3ms/step - loss: 0.1160 - accuracy: 0.9667 - val_loss: 0.1680 - val_accuracy: 0.9400 - _timestamp: 1649986012.0000 - _runtime: 8.0000
Epoch 5/6
300/300 [==============================] - 1s 3ms/step - loss: 0.0819 - accuracy: 0.9768 - val_loss: 0.1705 - val_accuracy: 0.9380 - _timestamp: 1649986013.0000 - _runtime: 9.0000
Epoch 6/6
300/300 [==============================]

accuracy,▁▅▇▇██
epoch,▁▂▄▅▇█
loss,█▄▂▂▁▁
val_accuracy,▁▅▄▆▆█
val_loss,█▄▃▁▁▁
accuracy,0.98167
best_epoch,5
best_val_loss,0.16089
epoch,5
loss,0.06481
val_accuracy,0.952


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 1/6
300/300 [==============================] - 1s 3ms/step - loss: 0.4410 - accuracy: 0.8733 - val_loss: 0.2860 - val_accuracy: 0.9100 - _timestamp: 1649986025.0000 - _runtime: 6.0000
Epoch 2/6
300/300 [==============================] - 1s 3ms/step - loss: 0.2119 - accuracy: 0.9396 - val_loss: 0.2267 - val_accuracy: 0.9120 - _timestamp: 1649986026.0000 - _runtime: 7.0000
Epoch 3/6
300/300 [==============================] - 1s 3ms/step - loss: 0.1412 - accuracy: 0.9598 - val_loss: 0.2032 - val_accuracy: 0.9240 - _timestamp: 1649986027.0000 - _runtime: 8.0000
Epoch 4/6
300/300 [==============================] - 1s 3ms/step - loss: 0.1111 - accuracy: 0.9678 - val_loss: 0.1665 - val_accuracy: 0.9400 - _timestamp: 1649986028.0000 - _runtime: 9.0000
Epoch 5/6
300/300 [==============================] - 1s 3ms/step - loss: 0.0832 - accuracy: 0.9768 - val_loss: 0.1745 - val_accuracy: 0.9460 - _timestamp: 1649986028.0000 - _runtime: 9.0000
Epoch 6/6
300/300 [==============================]

accuracy,▁▅▇▇██
epoch,▁▂▄▅▇█
loss,█▄▂▂▁▁
val_accuracy,▁▁▄▇█▅
val_loss,█▅▃▁▁▂
accuracy,0.98167
best_epoch,3
best_val_loss,0.16649
epoch,5
loss,0.06388
val_accuracy,0.93


Performing another sweep using PyTorch

In [14]:
import os
import random

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# remove slow mirror from list of MNIST mirrors
torchvision.datasets.MNIST.mirrors = [mirror for mirror in torchvision.datasets.MNIST.mirrors
                                      if not mirror.startswith("http://yann.lecun.com")]

In [15]:
config = dict(
    epochs=5,
    classes=10,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.005,
    dataset="MNIST",
    architecture="CNN")

In [16]:
def model_pipeline(hyperparameters):

    # tell wandb to get started
    with wandb.init(project="pytorch-demo", config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config)
      print(model)

      # and use them to train the model
      train(model, train_loader, criterion, optimizer, config)

      # and test its final performance
      test(model, test_loader)

    return model

def make(config):
    # Make the data
    train, test = get_data(train=True), get_data(train=False)
    train_loader = make_loader(train, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)

    # Make the model
    model = ConvNet(config.kernels, config.classes).to(device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)
    
    return model, train_loader, test_loader, criterion, optimizer

def get_data(slice=5, train=True):
    full_dataset = torchvision.datasets.MNIST(root=".",
                                              train=train, 
                                              transform=transforms.ToTensor(),
                                              download=True)
    #  equiv to slicing with [::slice] 
    sub_dataset = torch.utils.data.Subset(
      full_dataset, indices=range(0, len(full_dataset), slice))
    
    return sub_dataset


def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

# Conventional and convolutional neural network

class ConvNet(nn.Module):
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

def train(model, loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (images, labels) in enumerate(loader):

            loss = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss, example_ct, epoch)


def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")
        
        wandb.log({"test_accuracy": correct / total})

    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")

In [17]:
model = model_pipeline(config)

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)


  0%|          | 0/1 [00:00<?, ?it/s]

Loss after 00960 examples: 2.308
Loss after 01960 examples: 2.323
Loss after 02960 examples: 2.331
Loss after 03960 examples: 2.287
Loss after 04960 examples: 2.288
Loss after 05960 examples: 2.280
Loss after 06960 examples: 2.286
Loss after 07960 examples: 2.362
Loss after 08960 examples: 2.296
Loss after 09960 examples: 2.329
Loss after 10960 examples: 2.292
Loss after 11960 examples: 2.316
Accuracy of the model on the 2000 test images: 9.45%



epoch,▁▁▁▁▁▁▁▁▁▁▁▁
loss,▃▅▅▂▂▁▂█▂▅▂▄
test_accuracy,▁
epoch,0
loss,2.31638
test_accuracy,0.0945


##References:
#####https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb

#####https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases_keras.ipynb

#####https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Simple_PyTorch_Integration.ipynb